### Vector2d Compatible

In [10]:
from array import array 
A = array('d',[1,2,3,4,5])
Aa = iter(A)
Aa.__next__()

1.0

In [13]:
Aa.__next__()

3.0

In [18]:
len(A)

5

In [21]:
from array import array
import reprlib #to produce limited-length representations
import math

class Vector:
    typecode = 'd' #d for double

    def __init__(self,components):
        self._components = array(self.typecode, components)
    
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) #get a limited-length representation
        components = components[components.find('['):-1] #remove the array('d', prefix and the trailing)
        return 'Vector({})'.format(components)
    
    def __str__(self):
        return str(tuple(self))

    def __eq__(self,other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.sqrt(sum(x*x for x in self))

    def __bool__(self):
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self,index):
        return self._components[index]

In [22]:
Vector([1,2,3,4]).__str__

<bound method Vector.__str__ of Vector([1.0, 2.0, 3.0, 4.0])>

In [23]:
v1 = Vector([3,4,5])
v1[0]

3.0

In [25]:
v1[:2] # We want it converting to Vector

array('d', [3.0, 4.0])

### A Slice-Aware \_\_getitem\_\_

In [26]:
import numbers
class Vector2(Vector):
    typecode = 'd' #d for double
    
    def __getitem__(self,index):
        cls = type(self)
        if isinstance(index,slice):    #if index is a slice
            return cls(self._components[index]) #invoke the class to build vector instance
        elif isinstance(index,numbers.Integral):
            return self._components[index]
        else:
            msg = '{cls.__name__} indices must be integers'
            raise TypeError(msg.format(cls=cls))

In [28]:
v7 = Vector2(range(7))
v7

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

In [29]:
v7[:5]

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

### Dynamic Attribute Access

In [44]:
from typing import Any


class Vector3(Vector2):
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0<=pos< len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute{!r}'
        raise AttributeError(msg.format(cls,name)) 

In [45]:
v = Vector3(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [46]:
v.x

0.0

In [47]:
v.x = 10

In [48]:
v.x

10

In [49]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [50]:
v.b

AttributeError: 'Vector3' object has no attribute'b'

In [71]:

class Vector3(Vector2):
    typecode = 'd'
    shortcut_names = 'xyzt'

    def __getattr__(self, name):
        cls = type(self)
        if len(name) == 1:
            pos = cls.shortcut_names.find(name)
            if 0<=pos< len(self._components):
                return self._components[pos]
        msg = '{.__name__!r} object has no attribute{!r}'
        raise AttributeError(msg.format(cls,name)) 
    def __setattr__(self,name,value):
        cls = type(self)
        if len(name) == 1:
            if name in cls.shortcut_names:
                error = 'readonly attribute {attr_name!r}'
            elif name.islower():
                error = "can't set attributes 'a' to 'z' in {cls_name!r}"
            else:
                error = ''
            if error:
                msg = error.format(cls_name=cls.__name__, attr_name=name)
                raise AttributeError(msg)
        super().__setattr__(name, value)

In [72]:
v = Vector3(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [73]:
v.x

0.0

In [74]:
v.x=10

AttributeError: readonly attribute 'x'

### Hashing and a Faster ==

In [75]:
import functools
import operator
class Vector4(Vector3):
    typecode = 'd'
    shortcut_names = 'xyzt'
    def __hash__(self):
        hash = (hash(x) for x in self._components)
        return functools.reduce(operator.xor,hash,0) 
    def __eq__(self,other):
        return len(self) == len(other) and all(a == b for a,b in zip(self,other))